<a href="https://colab.research.google.com/github/NattachaiWat/ml442/blob/master/Lab5_Servering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np

In [0]:
from google.colab import files
print('Upload the model (flower_model.tar.gz) from Lab3')
model_gz = files.upload()

In [0]:
model_filename = list(model_gz.keys())[0]
!tar -xzf {model_filename} 

In [0]:
!saved_model_cli show --dir ./tmp/for_server/1  --tag_set serve --signature_def serving_default

In [0]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

In [0]:
!apt-get install tensorflow-model-server

In [0]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=flower \
  --model_base_path="/content/tmp/for_server" >server2.log 2>&1

In [0]:
!tail server2.log

In [0]:
import json
import numpy as np
import requests
from google.colab import files
from tensorflow.keras.preprocessing import image
import time

def show(img ,title):
  plt.figure()
  plt.imshow(img)
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

uploaded = files.upload()
class_name=['daisy','dandelion','roses','sunflowers','tulips']

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  images = images/225.0
  data = json.dumps({"signature_name": "serving_default", "instances": images[:].tolist()})
  headers = {"content-type": "application/json"}
  #time.sleep(2)
  json_response = requests.post('http://localhost:8501/v1/models/flower:predict', data=data, headers=headers)
  predictions = json.loads(json_response.text)['predictions']
  show(np.reshape(images,(224,224,3)),class_name[np.argmax(predictions,axis=1)[0]])


